In [1]:
import ROOT
import numpy as np
import toml
from tqdm.notebook import tqdm, trange
from utils import *
import numba

%jsroot on

Welcome to JupyROOT 6.28/00


## Load Config
The config holds all the information on how to construct the histograms and other general settings

In [2]:
config = toml.load("stage0_config.toml")

## Read Calibration
Read the detector calibration from csv file

In [3]:
file = open("calib/calib_apr.csv", "r")
file.readline()

numch = config['general']['numch']

calib_slope = [1] * numch
calib_offset = [0] * numch

for line in file:
    values = line.strip().split(',')
    calib_slope[int(values[0])] = float(values[1])
    calib_offset[int(values[0])] = float(values[2])

file.close()

calib_slope = np.array(calib_slope)
calib_offset = np.array(calib_offset)



## NUMBA Function
Create JIT compiled C++ function for creating hEgam column

In [4]:
@ROOT.Numba.Declare(["int","int"], "double")
def calc_Egam(detector,PulseHeight):
    return PulseHeight*calib_slope[detector]+calib_offset[detector]

## Create RDataFrame
Create data frame and define new columns then filter by channel. Create histogram models and then fill histograms.

In [5]:
ROOT.EnableImplicitMT()

tree_name = config['general']['tree_name']
numch = config['general']['numch']
fp_length = config['general']['fp_length']
pu_flag = config['general']['pu_flag']
fo_flag = config['general']['fo_flag']

file_name = "data/rawroot_run_0064*"

df = ROOT.RDataFrame(tree_name, file_name)

df = (df.Define("tof_ns","tof*10.0")
      .Define("tof_mus","tof_ns/1000.0+0.0000001")
      .Define("En","pow((72.3*21.5/(tof_ns/1000.0)),2)+0.00001")
      .Define("Egam","Numba::calc_Egam(detector,PulseHeight)"))

df_ch = []

for ch in range(numch):
    
    df_ch.append(df.Filter(f"detector == {ch}"))

hist_dict = {}

for key, hist_conf in config['hist'].items():
    
    # name = hist_conf['name']
    xaxis = hist_conf['xaxis']
    yaxis = hist_conf['yaxis']
    
    col = hist_conf['col']
    gate = hist_conf['gate']
    
    down = hist_conf['bins']['down']
    up = hist_conf['bins']['up']
    N = hist_conf['bins']['N']
    var = hist_conf['bins']['var']

    if hist_conf['all']:

        name = hist_conf['name']
        title  = name + ';' + xaxis + ';' + yaxis
        xbins = get_xbins(N, down, up, var, fp_length)

        hist_model_all = ROOT.RDF.TH1DModel(name, title, N, xbins)
        hist_all = df.Histo1D(hist_model_all, col)

        hist_dict[key] = hist_all

    else:
        hist_dict[key] = {}
        for ch in range(numch):
            
            name = hist_conf['name'] + f'_d{ch}'
            title  = name + ';' + xaxis + ';' + yaxis
            xbins = get_xbins(N, down, up, var, fp_length)

            hist_model_ch = ROOT.RDF.TH1DModel(name, title, N, xbins)
            hist_ch = df_ch[ch].Histo1D(hist_model_ch, col)

            hist_dict[key][name] = hist_ch
        
        
# hist_dict


## Write Histograms
Write histograms from dictionary to a root file maintaing heirerachy. 

In [6]:
write_dict_to_root(hist_dict,"test.root")

In [7]:
read_root_to_dict("test.root")

{'hTOF_mus': {'hTOF_mus_d0': <cppyy.gbl.TH1D object at 0x16e4a4fd0>,
  'hTOF_mus_d1': <cppyy.gbl.TH1D object at 0x16e49d660>,
  'hTOF_mus_d2': <cppyy.gbl.TH1D object at 0x16b3ccf80>,
  'hTOF_mus_d3': <cppyy.gbl.TH1D object at 0x16b3d9a70>,
  'hTOF_mus_d4': <cppyy.gbl.TH1D object at 0x16b3d9e60>,
  'hTOF_mus_d5': <cppyy.gbl.TH1D object at 0x16b3da250>,
  'hTOF_mus_d6': <cppyy.gbl.TH1D object at 0x16b3da640>,
  'hTOF_mus_d7': <cppyy.gbl.TH1D object at 0x16b3daa30>,
  'hTOF_mus_d8': <cppyy.gbl.TH1D object at 0x16b3dae20>,
  'hTOF_mus_d9': <cppyy.gbl.TH1D object at 0x107e97950>,
  'hTOF_mus_d10': <cppyy.gbl.TH1D object at 0x107e97f60>,
  'hTOF_mus_d11': <cppyy.gbl.TH1D object at 0x107e98350>,
  'hTOF_mus_d12': <cppyy.gbl.TH1D object at 0x107e98740>,
  'hTOF_mus_d13': <cppyy.gbl.TH1D object at 0x16b3db210>,
  'hTOF_mus_d14': <cppyy.gbl.TH1D object at 0x16b3db600>,
  'hTOF_mus_d15': <cppyy.gbl.TH1D object at 0x16b3db9f0>,
  'hTOF_mus_d16': <cppyy.gbl.TH1D object at 0x16b3dbde0>,
  'hTOF_mus_